In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV 

In [ ]:
import xgboost as xgb
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import time

In [ ]:
from xgboost import XGBClassifier

In [ ]:
import pickle

In [ ]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
import lime
from lime import lime_tabular

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [ ]:
data_xg_tsne1 = pd.read_csv('Cancer_3000features_SMOTE.csv')

In [ ]:
labels = data_xg_tsne1.target

In [ ]:
data_xg_tsne1.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
data_xg_tsne.shape

In [ ]:
data_xg_tsne1.head()

In [ ]:
X1 = data_xg_tsne1.iloc[:, :-1].values
y1 = data_xg_tsne1.iloc[:, -1].values

In [ ]:
y1

## PCA

In [ ]:
rndperm = np.random.permutation(data_xg_tsne.shape[0])

In [ ]:
pca = PCA(n_components=3)
pca_result = pca.fit_transform(data_xg_tsne.iloc[:,:-1].values)
data_xg_tsne['pca-one'] = pca_result[:,0]
data_xg_tsne['pca-two'] = pca_result[:,1] 
data_xg_tsne['pca-three'] = pca_result[:,2]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

with PdfPages(r'C:\Users\Shakshi Sharma\Downloads\PCA_RF.pdf') as export_pdf:

    plt.figure(figsize=(16,10))
    sns.scatterplot(
        x="pca-one", y="pca-two",
        hue=pd.Series(labels),
        palette=sns.color_palette("hls", pd.Series(labels).nunique()),
        data=data_xg_tsne.loc[rndperm,:],
        legend="full",
        alpha=1
    )
    export_pdf.savefig()
    plt.close()

In [ ]:
with PdfPages(r'C:\Users\Shakshi Sharma\Downloads\PCA_high_quality_after_SMOTE.pdf') as export_pdf:

    plt.figure(figsize=(16,10))
    sns.scatterplot(
        x="pca-one", y="pca-two",
        hue=pd.Series(labels),
        palette=sns.color_palette("hls", pd.Series(labels).nunique()),
        data=data_xg_tsne.loc[rndperm,:],
        alpha=1
    )
    plt.legend(bbox_to_anchor=(1,1.2), loc="upper left")
    #(bbox_to_anchor=(1.04,1), loc="upper left")
    plt.tight_layout()
    plt.gcf().subplots_adjust(bottom=0.04)

    export_pdf.savefig(bbox_inches='tight')

In [ ]:
with PdfPages(r'C:\Users\Shakshi Sharma\Downloads\PC_RF.pdf') as export_pdf:

    plt.figure(figsize=(16,10))
    sns.scatterplot(
        x="pca-one", y="pca-two",
        hue=pd.Series(labels),
        palette=sns.color_palette("hls", pd.Series(labels).nunique()),
        data=data_xg_tsne.loc[rndperm,:],
        alpha=1
    )
    
    plt.legend(loc="upper right", ncol=7)
    export_pdf.savefig()

In [ ]:
y_pred_prob = pd.read_csv('yy.csv')
y_pred_prob.drop('Unnamed: 0', axis = 1 , inplace = True)

In [ ]:
yy = pd.read_csv('yyy.csv')
yy.drop('Unnamed: 0', axis = 1 , inplace = True)

## T-SNE

In [ ]:
#targets = data500.target

In [ ]:
data_tsne = data_xg_tsne.copy()

In [ ]:
data_tsne.drop('target', axis = 1, inplace = True)

In [ ]:
time_start = time.time()
tsne = TSNE(n_components=3, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(data_tsne)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

In [ ]:
data_xg_tsne['tsne-2d-one'] = tsne_results[:,0]
data_xg_tsne['tsne-2d-two'] = tsne_results[:,1]
with PdfPages(r'C:\Users\Shakshi Sharma\Downloads\TSNE_high_quality_after_SMOTE.pdf') as export_pdf:

    plt.figure(figsize=(16,10))
    sns.scatterplot(
        x="tsne-2d-one", y="tsne-2d-two",
        hue=pd.Series(labels),
        palette=sns.color_palette("hls", pd.Series(labels).nunique()),
        data=data_xg_tsne,
        alpha=1
    )
    plt.legend(bbox_to_anchor=(1,1.2), loc="upper left")
    #(bbox_to_anchor=(1.04,1), loc="upper left")
    plt.tight_layout()
    plt.gcf().subplots_adjust(bottom=0.04)

    export_pdf.savefig(bbox_inches='tight')

In [ ]:
flag = True
while flag:
    X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size = 0.20, shuffle=True)
    if (pd.Series(y_test1).nunique() == pd.Series(y_train1).nunique()):
            flag = False

In [ ]:
from sklearn.preprocessing import StandardScaler
feature_scaler = StandardScaler()
X_train1 = feature_scaler.fit_transform(X_train1)
X_test1 = feature_scaler.transform(X_test1)

In [ ]:
# defining parameter range 
param_grid = {'max_depth': [3, 4, 5], 
            'n_estimators': [10, 50, 100, 500]} 
        
grid = GridSearchCV(XGBClassifier(), param_grid, scoring='accuracy',
                     cv=5, refit = True, verbose = 3, n_jobs=-1) 

# fitting the model for grid search 
grid.fit(X_train1, y_train1)

In [ ]:
print(grid.best_params_)

In [ ]:
folds = KFold(5,shuffle = True, random_state=0)
classifier = XGBClassifier(random_state = 42)

In [ ]:
for i, (train, test) in enumerate(folds.split(X_train1, y_train1)):

    classifier.fit(X_train1[train], y_train1[train])
    
    y_pred = classifier.predict(X_train1[test]) 
    conf_mat = confusion_matrix(y_train1[test], y_pred)
    print (conf_mat)

    precision = precision_score(y_train1[test], y_pred, average='micro')
    print('Precision micro: %.3f' % precision)
    precision = precision_score(y_train1[test], y_pred, average='macro')
    print('Precision macro: %.3f' % precision)
    recall = recall_score(y_train1[test], y_pred, average='micro')
    print('Recall micro: %.3f' % recall)
    recall = recall_score(y_train1[test], y_pred, average='macro')
    print('Recall macro: %.3f' % recall)
    print(accuracy_score(y_train1[test], y_pred))
    print(classification_report(y_train1[test], y_pred))

In [ ]:
y_pred_cv = classifier.predict(X_test1)
print(classification_report(y_test1, y_pred_cv))

In [ ]:
roc_auc_score(y_test1, classifier.predict_proba(X_test1), multi_class='ovr')

In [ ]:
precision_score(y_test1, y_pred_cv, average='micro')

In [ ]:
recall_score(y_test1, y_pred_cv, average='micro')

In [ ]:
f1_score(y_test1, y_pred_cv, average='micro')

In [ ]:
xgb.plot_importance(classifier)

In [ ]:
# save the model to disk
filename = 'after_smte_XG.sav'
pickle.dump(classifier, open(filename, 'wb'))

# some time later...

# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)

# LIME

In [ ]:
training_columns = data_xg.iloc[:, :-1]

In [ ]:
training_columns.columns

In [ ]:
explainer = lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train1),
    feature_names=training_columns.columns,
    class_names=list(np.unique(labels)),
    mode='classification'
)

In [ ]:
exp = explainer.explain_instance(
    data_row=X_test1[1], 
    predict_fn=classifier.predict_proba, num_features=10, top_labels=2
)

exp.show_in_notebook(show_table=True)

In [ ]:
exp = explainer.explain_instance(
    data_row=X_test1[2], 
    predict_fn=classifier.predict_proba, num_features=10, top_labels=2
)

exp.show_in_notebook(show_table=True)

In [ ]:
exp = explainer.explain_instance(
    data_row=X_test1[20], 
    predict_fn=classifier.predict_proba, num_features=10, top_labels=2
)

exp.show_in_notebook(show_table=True)